# Análisis y Diseño de Algoritmos
## Introducción a la programación con Backtracking en Python

En este notebook veremos cómo implementar en Python algoritmos mediante Backtracking (BT) siguiendo los esquemas y funciones descritas en el tema de teoría.

## Instalación e importación de librerías

Antes de nada, instalemos e importemos las librerías que vamos a necesitar.

In [ ]:
!sudo apt-get install graphviz libgraphviz-dev
!pip install pygraphviz

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

import matplotlib.pyplot as plt

print(nx.__version__)

## Métodos generales

En teoría hemos visto que el esquema general de BT cuando se sigue un esquema no recursivo es el siguiente:

In [3]:
def backtracking(s_inicial):
  nivel= 0
  s= s_inicial
  fin= False
  while fin != True:
    s= generar(nivel, s)
  if solucion(nivel, s):
    fin= True
  elif criterio(nivel, s):
    nivel= nivel + 1
  else:
    while not mas_hermanos(nivel, s):
      s= retroceder(nivel, s)
  return s

En este caso hemos encapsulado dicho esquema dentro de una función que hemos denominado `backtracking`. Dicha función toma como entrada `s_inicial` que constituye la configuración inicial de `s` que contendrá la solución generada por el sistema.

Es importante remarcar que, dependiendo del problema `s_inicial` y `s` podrán adoptar la forma de tupla, lista, diccionarios, etc. No debemos asumir que dichas variables tomarán siempre la misma forma independientemente del problema planteado.

Por último, el esquema depende de una serie de funciones auxilares (`generar`, `solucion`, `criterio`, `mas_hermanos` y `retroceder`) que pasamos a continuación a definir.

In [4]:
def generar(s):
  """
  Genera el siguiente hermano para el nivel actual.
  Al llegar a un nivel, “siguiente” = primero.​
  """
  pass
def solucion(c):
  """
  Comprueba si s es una solución válida para el problema.​
  """
  pass
def criterio(s,x):
  """
  Comprueba si a partir de solución parcial s puede haber solución válida.
  SI NO se rechazarán todos los descendientes
  """
  pass
def mas_hermanos (s,x):
  """
  Devuelve True si hay más hermanos de nodo actual (s[nivel]) aún no generados
  """
  pass
def retroceder(s):
  """
  Retrocede un nivel en el árbol de soluciones.
  Disminuye en 1 el valor de nivel, y actualiza solución
  actual y otras variables, quitando elementos 	retrocedidos y
  “dejando todo como estaba”.  """
  pass

### Variaciones del esquema general

En teoría hemos visto también que existen diferentes *variaciones* de dicho esquema general en función de las características del problema al cual nos enfrentemos.



#### Esquema cuando puede no haber solución

Así, si nos enfrentamos a un problema para el cual puede que no exista una solución válida deberemos usar y adaptar este esquema que hemos definido como la función `backtracking_no_solucion`:

In [5]:
def backtracking_no_solucion(s_inicial):
  nivel= 0
  s= s_inicial
  fin= False
  while not fin and nivel !=0:
    s= generar(nivel, s)
    if solucion(nivel, s):
      fin= True
    elif criterio(nivel, s):
      nivel= nivel + 1
    else:
      while not mas_hermanos (nivel, s) and nivel >0:
        s= retroceder(nivel, s)
  return s

#### Esquema cuando queremos todas las soluciones

Por el contrario, si queremos obtener todas las soluciones posibles de un problema (porque sabemos que puede haber más de una solución válida), tenemos este esquema que hemos definido como la función `backtracking_todas`:

In [6]:
def backtracking_todas(s_inicial):
  nivel= 0
  s= s_inicial
  s_todas= []
  fin= False
  while nivel !=0:
    s= generar(nivel, s)
    if solucion(nivel, s):
      s_todas.append(almacenar(nivel, s))
    if criterio(nivel, s):
      nivel= nivel + 1
    else:
      while not mas_hermanos (nivel, s) and nivel >0:
        s= retroceder(nivel, s)
  return s_todas

Como vemos este esquema implica una nueva función auxiliar `almacenar` para almacenar cada una de las soluciones encontradas.

In [7]:
def almacenar(nivel, s):
  """
  Registra la solución contenida en s como una de las encontradas dentro de un nivel del arbol
  """
  pass

#### Esquema cuando queremos la mejor solución (optimización)

Por último, tenemos el esquema cuando nuestro propósito es obtener la mejor solución posible que maximice/minimice un determinado objetivo.

En este caso, tenemos el esquema contenido dentro de la función `backtracking_opt`

In [8]:
def backtracking_opt(s_inicial):
  nivel= 1
  s= s_inicial
  voa= float('-inf')
  soa= None
  while nivel !=0:
    s= generar(nivel, s)
    if solucion(nivel, s) and valor(s) > voa:
      voa= valor(s)
      soa= s
    if criterio(nivel, s):
      nivel= nivel + 1
    else:
      while not mas_hermanos(nivel, s) and nivel >0:
        s= retroceder(nivel, s)
  return voa,soa

De nuevo esto nos obliga a definir una función nueva `valor`.

In [9]:
def valor(s):
  """
  Devuelve el valor asociado a la variable solucion s
  """
  pass

### Ejemplo 1: Encontrar el subconjunto que sume P.

Como primer ejemplo vamos a implementar en Python la solución del problema planteado en clase de teoría de encontrar un subconjunto del conjunto $T= \{t_1, t_2, ..., t_n\}$ que sume exactamente una cantidad `P`.

Para ello vamos a seguir los siguientes pasos:

#### Paso 1: Definir la variable solucion `s`.

En primer lugar debemos definir qué forma quiero que adopte la variable `s` que almacenerá la solución del problema.

En este caso, tal y como vimos en teoría, podemos asumir una lista de `n` elementos (tantos como el conjunto inicial `T`) donde cada elemento de `s` podrá tomar tres valores diferentes:

- 0: el número i-ésimo no se utiliza.
- 1: el número i-ésimo sí se utiliza.
- -1: valor de inicialización

Puesto que la función `backtracking` que hemos definido antes toma com parámetro de entrada la variable `s_inicial` con los valores iniciales que debe de tomar `s`, podemos ya inicializar dicha variable como una lista de longitud `n` en donde todos sus elementos son -1.

In [ ]:
T= (4,2,7,8) # Asumimos que el conjunto de valores es este.

s_inicial = [-1] * len(T)
print(s_inicial)

#### Paso 2: Definir el esquema BT.

Una vez ya hemos fijado cual va a ser la estructura de la solución que queremos generar, lo siguiente será decidir qué esquema BT elegir de entre todos los disponibles.

En este caso no es seguro que podemos encontrar una solución al problema planteado aunque nos valdría cualquier solución (si existiera). Por tanto, lo más razonable sería re-usar la implementación de la función `bactracking_no_solución`.

Podemos optimizar dicha implementación, pasando la lista `T` de números como parámetro de entrada e inicializando `s` directamente. Además, deberemos pasar también como parámetro de entrada del problema la cantidad `p` que queremos obtener. Vamos también a renombrar las funciones para asignarlas a este problema.

In [11]:
def backtracking_no_solucion_subconjunto(T:list, p:float):
  nivel= 0
  s=  [-1] * len(T) #inicializamos s (nos ahorramos generar s_inicial)
  num_elementos = len(T)
  fin= False
  t_act = 0
  while not fin and nivel !=-1:
    s, t_act= generar_subconjunto(nivel, s, t_act, T)
    print(f"s:{s}, t_act: {t_act}")
    if solucion_subconjunto(nivel, num_elementos, t_act, p):
      print("Encontrada una solución!")
      fin= True
    elif criterio_subconjunto(nivel, num_elementos, t_act, p):
      nivel= nivel + 1
    else:
      while not mas_hermanos_subconjunto(nivel, s) and nivel >= 0:
        s, t_act, nivel= retroceder_subconjunto(nivel, s, t_act, T)
  return s

#### Paso 3: Implementar las funciones auxiliares.

Podemos ahora implementar las funciones auxiliares siguiendo la propuesta vista en teoría.

*NOTA: Cuidado, pues la numeración en Python comienza en 0*

In [12]:
def generar_subconjunto(nivel:int, s:list, t_act:float, T:list):
  s[nivel]= s[nivel] + 1
  if s[nivel]==1:
    t_act= t_act + T[nivel]
  return s, t_act

def solucion_subconjunto(nivel:int, num_elementos:int, t_act:float, p:float):
  return (nivel==(num_elementos-1)) and (t_act==p)

def criterio_subconjunto(nivel:int, num_elementos:int, t_act:float, p:float):
  return (0 <= nivel< (num_elementos-1)) and (t_act<= p)

def mas_hermanos_subconjunto(nivel:int, s:list):
  return s[nivel]< 1

def retroceder_subconjunto(nivel:int, s:list, t_act:float, T:list):
  t_act= t_act - T[nivel]*s[nivel]
  s[nivel]= -1
  nivel= nivel- 1
  return s, t_act, nivel

#### Paso 4: Invocar el algoritmo principal

Ahora ya estamos en condiciones de invocar a nuestra función `backtracking_no_solucion_subconjunto` indicando los parámetros del modelo.

Para ello, vamos a juntar todo el código que hemos desarrollado en una única celda.


In [13]:
def backtracking_no_solucion_subconjunto(T:list, p:float):
  nivel= 0
  s=  [-1] * len(T) #inicializamos s (nos ahorramos generar s_inicial)
  num_elementos = len(T)
  fin= False
  t_act = 0
  while not fin and nivel !=-1:
    s, t_act= generar_subconjunto(nivel, s, t_act, T)
    print(f"s:{s}, t_act: {t_act}")
    if solucion_subconjunto(nivel, num_elementos, t_act, p):
      print("Encontrada una solución!")
      fin= True
    elif criterio_subconjunto(nivel, num_elementos, t_act, p):
      nivel= nivel + 1
    else:
      while not mas_hermanos_subconjunto(nivel, s) and nivel >= 0:
        s, t_act, nivel= retroceder_subconjunto(nivel, s, t_act, T)
  return s

def generar_subconjunto(nivel:int, s:list, t_act:float, T:list):
  s[nivel]= s[nivel] + 1
  if s[nivel]==1:
    t_act= t_act + T[nivel]
  return s, t_act

def solucion_subconjunto(nivel:int, num_elementos:int, t_act:float, p:float):
  return (nivel==(num_elementos-1)) and (t_act==p)

def criterio_subconjunto(nivel:int, num_elementos:int, t_act:float, p:float):
  return (0 <= nivel< (num_elementos-1)) and (t_act<= p)

def mas_hermanos_subconjunto(nivel:int, s:list):
  return s[nivel]< 1

def retroceder_subconjunto(nivel:int, s:list, t_act:float, T:list):
  t_act= t_act - T[nivel]*s[nivel]
  s[nivel]= -1
  nivel= nivel- 1
  return s, t_act, nivel

Primero vamos a pasarle un configuración de parámetros para el cual hay solucíón.

In [ ]:
T= (4,2,7,8) # Asumimos que el conjunto de valores es este.
p= 9

sol_ = backtracking_no_solucion_subconjunto(T, p)
print(sol_)

Probemos ahora con una configuración para la cual no hay solución

In [ ]:
T= (4,2,7,8)
p= 5

sol_ = backtracking_no_solucion_subconjunto(T, p)
print(sol_)

#### BONUS: Añadir mecanismo de visualización del árbol

Un elemento interesante a incorporar en la implementación de un algoritmo de BT es la visualización del árbol de computación generado por el algoritmo.

Nosotros vamos a modificar el algoritmo anterior para incorporar dicho mecanismo. Para ello haremos uso de la librería [`networkx`](https://networkx.org/).

Brevemente, definimos una nueva variable  `G` que irá almacenando los nodos y enlaces del arbol y una variable `rama_actual` en donde almacenaremos los nodos de la rama activa del algoritmo.


In [16]:
def backtracking_no_solucion_subconjunto_con_grafo(T:list, p:float):
  nivel= 0
  s=  [-1] * len(T) #inicializamos s (nos ahorramos generar s_inicial)
  num_elementos = len(T)
  fin= False
  t_act = 0

  G= nx.Graph() #Generamos el grafo vacío.
  nodo_actual_id = 0 #Identificamos numericamente a los nodos
  rama_actual= [-1] * (len(T)+1) #Almacenamos la información de la rama actual
  rama_actual[nivel]= nodo_actual_id
  G.add_node(nodo_actual_id, t_act=t_act)

  while not fin and nivel !=-1:
    s, t_act= generar_subconjunto(nivel, s, t_act, T)
    nodo_actual_id += 1 #Creamos un nuevo nodo
    rama_actual[nivel+1]= nodo_actual_id
    G.add_node(nodo_actual_id, t_act=t_act)

    if (nivel+1) > 0:
      G.add_edge(rama_actual[nivel], rama_actual[nivel+1], value=s[nivel])  # Agregar una arista

    print(f"s:{s}, t_act: {t_act}")
    if solucion_subconjunto(nivel, num_elementos, t_act, p):
      print("Encontrada una solución!")
      fin= True
    elif criterio_subconjunto(nivel, num_elementos, t_act, p):
      nivel= nivel + 1
    else:
      while not mas_hermanos_subconjunto(nivel, s) and nivel >=0:
        s, t_act, nivel= retroceder_subconjunto(nivel, s, t_act, T)
  return s, G


def generar_subconjunto(nivel:int, s:list, t_act:float, T:list):
  s[nivel]= s[nivel] + 1
  if s[nivel]==1:
    t_act= t_act + T[nivel]
  return s, t_act

def solucion_subconjunto(nivel:int, num_elementos:int, t_act:float, p:float):
  return (nivel==(num_elementos-1)) and (t_act==p)

def criterio_subconjunto(nivel:int, num_elementos:int, t_act:float, p:float):
  return (0 <= nivel< (num_elementos-1)) and (t_act<= p)

def mas_hermanos_subconjunto(nivel:int, s:list):
  return s[nivel]< 1

def retroceder_subconjunto(nivel:int, s:list, t_act:float, T:list):
  t_act= t_act - T[nivel]*s[nivel]
  s[nivel]= -1
  nivel= nivel- 1
  return s, t_act, nivel

Implementamos la función externa para visualizar el grafo generado por BT.

In [17]:
def show_bt_graph(G:nx.Graph):
  pos = graphviz_layout(G, prog='dot')

  # Obtener etiquetas de nodos (incluyendo t_act)
  node_labels = {node: f"{node}\n t_act={data['t_act']}" for node, data in G.nodes(data=True)}

  # Obtener etiquetas de aristas (ej. peso y otro atributo)
  edge_labels = {(u, v): f"{data['value']}" for u, v, data in G.edges(data=True)}

  # Dibujar el grafo
  plt.figure(figsize=(6, 6))
  nx.draw(G, pos, with_labels=False, node_color="lightblue", node_size=2000, edge_color="gray")
  nx.draw_networkx_labels(G, pos, node_labels, font_size=10, font_color="black")

  # Dibujar etiquetas de aristas
  nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

  plt.show()

Probemos ahora el nuevo código con las dos configuraciones de entrada que hemos probado antes.

In [ ]:
T= (4,2,7,8) # Asumimos que el conjunto de valores es este.
p= 9

sol_, G = backtracking_no_solucion_subconjunto_con_grafo(T, p)
print(sol_)
show_bt_graph(G)

In [ ]:
T= (4,2,7,8) # Asumimos que el conjunto de valores es este.
p= 5

sol_, G = backtracking_no_solucion_subconjunto_con_grafo(T, p)
print(sol_)
show_bt_graph(G)

### Ejemplo 2: Asignación de tareas

Implementemos ahora el otro de los ejemplos de BT vistos en teoría.

- Existen n personas y n trabajos.

- Cada persona *i*  puede realizar un trabajo *j*  con más o menos rendimiento: `B[i, j]`.

- **Objetivo**: asignar una tarea a cada trabajador (asignación uno-a-uno), de manera que se maximice la suma de rendimientos

#### Paso 1: Definir la variable solución `s`

En este caso vamos a definir la variable `s` como una `list` de enteros en donde `s[i]` indica la tarea asignada a la persona `i`.

Esto dará lugar a un árbol permutacional *¡¡Repasa los apuntes de teoría si no te acuerdas!!* ;-)

In [ ]:
s= []


#### Paso 2: Definir el esquema de BT.

En este caso deberemos hacer uso de la variante de BT para problemas de optimización pues nuestro propósito es *maximizar* la suma de rendimientos.

Por tanto, deberemos de tomar como referencia la implementación de la funcion `backtrackint_opt` que vimos antes.

En este sentido vamos a usar la lista `usada` vista en clase para optimizar el funcionamiento del algoritmo.

In [20]:
def backtracking_opt(s_inicial):
  nivel= 0
  s= s_inicial
  voa= float('-inf')
  soa= None
  while nivel != -1:
    s= generar(nivel, s)
    if solucion(nivel, s) and valor(s) > voa:
      voa= valor(s)
      soa= s
    if criterio(nivel, s):
      nivel= nivel + 1
    else:
      while not mas_hermanos(nivel, s) and nivel >0:
        s= retroceder(nivel, s)
  return voa,soa

#### Paso 3: Implementar las funciones auxiliares

Pasemos a implementar todas las funciones siguiendo lo visto en teoría.

In [21]:
def generar_asignacion(nivel:int, s:list, B:list, usada:list, bact:float): #Probar primero 1, luego 2, ..., ​
  if (s[nivel] >= 0) and (usada[s[nivel]] != 0):
    usada[s[nivel]] -= 1
  s[nivel]= s[nivel] + 1
  usada[s[nivel]] += 1

  if s[nivel]==0:
    bact= bact + B[nivel][s[nivel]]
  else:
    bact= bact + B[nivel][s[nivel]] - B[nivel][s[nivel]-1]
  return s, bact, usada

def criterio_asignacion(nivel:int, s:list, n:int, usada:list):
  return (usada[s[nivel]]==1) and (nivel < (n-1))

def solucion_asignacion(nivel:int, s:list, n:int, usada:list):
  return (usada[s[nivel]]==1) and (nivel == (n-1))

def mas_hermanos_asignacion(nivel:int, s:list, n:int):
  return s[nivel] < (n-1)

def retroceder_asignacion(nivel:int, s:list, B:list, usada:list, bact:float):

  bact= bact - B[nivel][s[nivel]]
  usada[s[nivel]] -= 1
  s[nivel]= -1
  nivel= nivel - 1

  return nivel, s, bact, usada

def valor_asignacion(s:list, B:list):
  valor= 0
  for i in range(s):
    j= s[i]
    valor += B[i,j]
  return valor

#### Paso 4: Invocar el algoritmo principal

Las funciones auxiliares que hemos implementado necesita de algunos parámetros de entrada y salidas extras respecto a la implementación base, por lo que necesitamos adaptar el algoritmo principal

In [22]:
def generar_asignacion(nivel:int, s:list, B:list, usada:list, bact:float): #Probar primero 1, luego 2, ..., ​
  if (s[nivel] >= 0) and (usada[s[nivel]] != 0):
    usada[s[nivel]] -= 1
  s[nivel]= s[nivel] + 1
  usada[s[nivel]] += 1

  if s[nivel]==0:
    bact= bact + B[nivel][s[nivel]]
  else:
    bact= bact + B[nivel][s[nivel]] - B[nivel][s[nivel]-1]
  return s, bact, usada

def criterio_asignacion(nivel:int, s:list, n:int, usada:list):
  return (usada[s[nivel]]==1) and (nivel < (n-1))

def solucion_asignacion(nivel:int, s:list, n:int, usada:list):
  return (usada[s[nivel]]==1) and (nivel == (n-1))

def mas_hermanos_asignacion(nivel:int, s:list, n:int):
  return s[nivel] < (n-1)

def retroceder_asignacion(nivel:int, s:list, B:list, usada:list, bact:float):

  bact= bact - B[nivel][s[nivel]]
  usada[s[nivel]] -= 1
  s[nivel]= -1
  nivel= nivel - 1

  return nivel, s, bact, usada

def valor_asignacion(s:list, B:list):
  valor= 0
  for i in range(s):
    j= s[i]
    valor += B[i,j]
  return valor

"""
Aqui tenemos nuestra función principal
"""

def backtracking_opt_asignacion(B:list):
  print("*"*5, "Iniciando BT...")
  nivel= 0
  s= [-1] * len(B) #asumimos que las filas son los trabajadores
  voa= float('-inf')
  soa= None
  num_tareas= len(B[0]) #asumimos que las columnas son las
  num_trabajadores= len(B)
  usada = [0] * num_tareas #indica el número de veces que una tarea ha sido usada
  bact= 0
  while nivel != -1:
    s, bact, usada= generar_asignacion(nivel, s, B, usada, bact)
    if solucion_asignacion(nivel, s, num_trabajadores, usada) and bact > voa:
      print(f"Nueva mejor asignacion encontrada: {s}, {bact}")
      voa= bact
      soa= s.copy() #Recuerda que en Python las asignaciones de listas son por referencia por defecto.
    if criterio_asignacion(nivel, s, num_trabajadores, usada):
      nivel= nivel + 1
    else:
      while not mas_hermanos_asignacion(nivel, s, num_tareas) and nivel >= 0:
        nivel, s, bact, usada= retroceder_asignacion(nivel, s, B, usada, bact)
  print("*"*5, "Terminando BT...")
  return voa, soa


In [ ]:
B= [[4,9,1],[7,2,3],[6,3,5]] #3 tareas y 3 trabajadores

beneficio, asignacion= backtracking_opt_asignacion(B)

print("La mejor asignación de tareas es la siguiente:")
for i in range(len(asignacion)):
  print(f"\t Para el trabajador {i} la tarea {asignacion[i]}")
print(f"Con esto obtenemos un beneficio de {beneficio}")


#### BONUS: Añadir mecanismo de visualización del árbol

Mejoremos el código para visualizar el árbol generado por nuestro algoritmo.

*Ojo pues tenemos que tener cuidado en registrar en el arbol todos los nodos correctos*

In [24]:
def backtracking_opt_asignacion_con_grafo(B:list):
  print("*"*5, "Iniciando BT...")

  nivel= 0
  s= [-1] * len(B) #asumimos que las filas son los trabajadores
  voa= float('-inf')
  soa= None
  num_tareas= len(B[0]) #asumimos que las columnas son las
  num_trabajadores= len(B)
  usada = [0] * num_tareas #indica el número de veces que una tarea ha sido usada
  bact= 0

  G= nx.Graph() #Generamos el grafo vacío.
  nodo_actual_id = 0 #Identificamos numericamente a los nodos
  rama_actual= [-1] * (len(B)+1) #Almacenamos la información de la rama actual
  rama_actual[nivel]= nodo_actual_id
  G.add_node(nodo_actual_id, b_act=bact)


  while nivel != -1:
    s, bact, usada= generar_asignacion(nivel, s, B, usada, bact)

    if solucion_asignacion(nivel, s, num_trabajadores, usada):

      nodo_actual_id += 1 #Creamos un nuevo nodo
      rama_actual[nivel+1]= nodo_actual_id
      G.add_node(nodo_actual_id, b_act=bact)

      if (nivel+1) > 0:
        G.add_edge(rama_actual[nivel], rama_actual[nivel+1], value=s[nivel])  # Agregar una arista

      if bact > voa:
        print(f"Nueva mejor asignacion encontrada: {s}, {bact}")
        voa= bact
        soa= s.copy() #Recuerda que en Python las asignaciones de listas son por referencia por defecto.
    if criterio_asignacion(nivel, s, num_trabajadores, usada):

      nodo_actual_id += 1 #Creamos un nuevo nodo
      rama_actual[nivel+1]= nodo_actual_id
      G.add_node(nodo_actual_id, b_act=bact)

      if (nivel+1) > 0:
        G.add_edge(rama_actual[nivel], rama_actual[nivel+1], value=s[nivel])  # Agregar una arista

      nivel= nivel + 1
    else:
      while not mas_hermanos_asignacion(nivel, s, num_tareas) and nivel >= 0:
        nivel, s, bact, usada= retroceder_asignacion(nivel, s, B, usada, bact)
  print("*"*5, "Terminando BT...")
  return voa, soa, G


Implementamos también una función de visualización de árbol generado. Es básicamente la misma función que generamos en el anterior caso pero modificando unos pequeños parámetros internos (tamaño de `figsize` y cambiar `t_act` por `b_act`).

In [25]:
def mostrar_arbol_bt_asignacion(G:nx.Graph):
  pos = graphviz_layout(G, prog='dot')

  # Obtener etiquetas de nodos (incluyendo t_act)
  node_labels = {node: f"{node}\n b_act={data['b_act']}" for node, data in G.nodes(data=True)}

  # Obtener etiquetas de aristas (ej. peso y otro atributo)
  edge_labels = {(u, v): f"{data['value']}" for u, v, data in G.edges(data=True)}

  # Dibujar el grafo
  plt.figure(figsize=(18, 6))
  nx.draw(G, pos, with_labels=False, node_color="lightblue", node_size=2000, edge_color="gray")
  nx.draw_networkx_labels(G, pos, node_labels, font_size=10, font_color="black")

  # Dibujar etiquetas de aristas
  nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)
  plt.show()

Ahora podemos invocar a la nueva versión mejorada del algoritmo


In [ ]:
B= [[4,9,1],[7,2,3],[6,3,5]] #3 tareas y 3 trabajadores

beneficio, asignacion, arbol= backtracking_opt_asignacion_con_grafo(B)

print("La mejor asignación de tareas es la siguiente:")
for i in range(len(asignacion)):
  print(f"\t Para el trabajador {i} la tarea {asignacion[i]}")
print(f"Con esto obtenemos un beneficio de {beneficio}")


In [ ]:
mostrar_arbol_bt_asignacion(arbol)

### Ejercicios

### Ejercicio 1: Modifica la función `criterio_subconjunto` para incluir un mecanismo de poda que permita no seguir avanzando en el árbol si la suma de los elementos que quedan por considerar más el valor `t_act` ya es menor al valor `p`

### Ejercicio 3: Modifica las funciones auxiliares necesarias para que el algoritmo de asignación de tareas pueda asignar `num_rep` veces diferentes una misma tarea.

In [ ]:
print("¡Eso es todo amigos!")